In [1]:
pip install eikon

Note: you may need to restart the kernel to use updated packages.


In [38]:
import eikon as ek
import numpy as np
import pandas as pd
import datetime
from datetime import datetime
import math
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import scipy
import seaborn as sns
import xlsxwriter
import math
from math import exp, sqrt, log
%matplotlib inline
from scipy import stats
import statsmodels as sm
from statsmodels.regression.linear_model import OLS
import sqlite3
from sqlite3 import Error

from pandas_datareader import data as pdr
import plotly.offline as pyo
import plotly.graph_objects as go
from plotly.subplots import make_subplots

pyo.init_notebook_mode(connected=True)
pd.options.plotting.backend ='plotly'

In [3]:
ek.set_app_key('dfd3e4f6ef7143e8bf90ac68e33c1499304694ed')

In [4]:
#creating list containing RICs of analysed companies
rics = ["JPM.N","BAC.N","C.N","WFC.N", "GS","MS.N","HSBA.L","BNPP.PA","CAGR.PA","SAN.MC","SOGN.PA","BARC.L"]
 
#retrieving the Close Price and the Date for the last 10 years for all RICs
price_close = ek.get_data(rics,['TR.ClosePrice.Date','TR.ClosePrice'], {'Sdate':'2002-01-01', 'EDate':'2021-12-31'})
price_close

(      Instrument                  Date  Close Price
 0          JPM.N  2002-01-01T00:00:00Z         <NA>
 1          JPM.N  2002-01-02T00:00:00Z        36.35
 2          JPM.N  2002-01-03T00:00:00Z        37.32
 3          JPM.N  2002-01-04T00:00:00Z         39.0
 4          JPM.N  2002-01-07T00:00:00Z         38.9
 ...          ...                   ...          ...
 60794     BARC.L  2021-12-23T00:00:00Z        187.4
 60795     BARC.L  2021-12-24T00:00:00Z       187.44
 60796     BARC.L  2021-12-29T00:00:00Z        188.5
 60797     BARC.L  2021-12-30T00:00:00Z       188.06
 60798     BARC.L  2021-12-31T00:00:00Z        187.0
 
 [60799 rows x 3 columns],
 None)

In [5]:
#grouping the results by RICs and concating all df
dfs = dict(tuple(price_close[0].groupby('Instrument'))) 
dfarray = [] 
for ric, data in dfs.items(): 
   df_tmp = dfs[ric].dropna() 
   df_tmp = df_tmp.drop_duplicates() 
   df_tmp = df_tmp.set_index('Date')
   df_tmp = df_tmp.drop(['Instrument'], axis=1) 
   df_tmp = df_tmp.rename(columns={"Close Price":ric}) 
   dfarray.append(df_tmp) 
     
price_close_converted= pd.concat(dfarray, axis=1, sort=True) 
price_close_converted.columns.name = 'CLOSE'

#creating a dataframe, changing the inxed to Datetime and droping the Timestamp
df_price_close = pd.DataFrame(price_close_converted)
df_price_close.index = pd.to_datetime(df_price_close.index).strftime('%Y-%m-%d')
df_price_close

CLOSE,BAC.N,BARC.L,BNPP.PA,C.N,CAGR.PA,GS,HSBA.L,JPM.N,MS.N,SAN.MC,SOGN.PA,WFC.N
Date,,,,,,,,,,,,
2002-01-02,31.48,509.406461,48.362064,510.1,15.947297,91.07,706.734596,36.35,46.168062,5.593177,55.256933,21.72
2002-01-03,31.445,514.579242,48.844238,510.5,15.974326,92.45,724.163316,37.32,47.505782,5.676303,56.402611,21.8
2002-01-04,31.935,517.727891,49.229977,520.0,15.938287,96.75,731.134804,39.0,49.863513,5.711928,58.209257,21.855
2002-01-07,31.555,511.430593,49.085325,514.3,16.379766,95.75,724.163316,38.9,49.487279,5.462551,56.622934,21.89
2002-01-08,31.035,509.181558,49.18176,495.0,16.668079,92.5,716.320392,38.6,48.433825,5.545676,57.107643,21.805
...,...,...,...,...,...,...,...,...,...,...,...,...
2021-12-27,44.64,<NA>,60.38,60.65,12.504,388.04,<NA>,158.16,100.4,2.9235,29.99,48.77
2021-12-28,44.7,<NA>,60.82,60.62,12.55,387.61,<NA>,158.64,99.97,2.931,30.145,48.42
2021-12-29,44.63,188.5,60.71,60.54,12.568,386.2,449.4,158.56,98.73,2.9225,30.275,48.45


In [6]:
#checking how many NAs are in each column to get a feeling on how many rows will be deleted
df_price_close.isna().sum(axis = 0)

CLOSE
BAC.N      131
BARC.L     111
BNPP.PA     45
C.N        131
CAGR.PA     45
GS         131
HSBA.L     111
JPM.N      131
MS.N       131
SAN.MC      75
SOGN.PA     45
WFC.N      131
dtype: int64

In [7]:
#droping rows with NAs in it
df_price_close_corr = df_price_close.dropna()
df_price_close_corr

CLOSE,BAC.N,BARC.L,BNPP.PA,C.N,CAGR.PA,GS,HSBA.L,JPM.N,MS.N,SAN.MC,SOGN.PA,WFC.N
Date,,,,,,,,,,,,
2002-01-02,31.48,509.406461,48.362064,510.1,15.947297,91.07,706.734596,36.35,46.168062,5.593177,55.256933,21.72
2002-01-03,31.445,514.579242,48.844238,510.5,15.974326,92.45,724.163316,37.32,47.505782,5.676303,56.402611,21.8
2002-01-04,31.935,517.727891,49.229977,520.0,15.938287,96.75,731.134804,39.0,49.863513,5.711928,58.209257,21.855
2002-01-07,31.555,511.430593,49.085325,514.3,16.379766,95.75,724.163316,38.9,49.487279,5.462551,56.622934,21.89
2002-01-08,31.035,509.181558,49.18176,495.0,16.668079,92.5,716.320392,38.6,48.433825,5.545676,57.107643,21.805
...,...,...,...,...,...,...,...,...,...,...,...,...
2021-12-21,44.19,184.26,58.86,59.38,12.174,380.32,445.2,156.09,97.77,2.811,29.12,47.81
2021-12-22,44.28,185.84,59.35,59.6,12.252,382.33,444.6,156.7,98.3,2.8155,29.44,48.03
2021-12-23,44.42,187.4,60.0,60.21,12.44,385.04,447.9,157.26,99.34,2.9015,29.83,48.36


In [8]:
#double-checking
df_price_close_corr.isna().sum(axis = 0)

CLOSE
BAC.N      0
BARC.L     0
BNPP.PA    0
C.N        0
CAGR.PA    0
GS         0
HSBA.L     0
JPM.N      0
MS.N       0
SAN.MC     0
SOGN.PA    0
WFC.N      0
dtype: int64

In [9]:
# adding date column to dataframe
index1=df_price_close_corr.index
df_price_close_corr['Date1']=index1
df_price_close_corr.head(5)

C:\Users\Besitzer\AppData\Local\Temp/ipykernel_10084/2239153136.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_price_close_corr['Date1']=index1


CLOSE,BAC.N,BARC.L,BNPP.PA,C.N,CAGR.PA,GS,HSBA.L,JPM.N,MS.N,SAN.MC,SOGN.PA,WFC.N,Date1
Date,,,,,,,,,,,,,
2002-01-02,31.48,509.406461,48.362064,510.1,15.947297,91.07,706.734596,36.35,46.168062,5.593177,55.256933,21.72,2002-01-02
2002-01-03,31.445,514.579242,48.844238,510.5,15.974326,92.45,724.163316,37.32,47.505782,5.676303,56.402611,21.8,2002-01-03
2002-01-04,31.935,517.727891,49.229977,520.0,15.938287,96.75,731.134804,39.0,49.863513,5.711928,58.209257,21.855,2002-01-04
2002-01-07,31.555,511.430593,49.085325,514.3,16.379766,95.75,724.163316,38.9,49.487279,5.462551,56.622934,21.89,2002-01-07
2002-01-08,31.035,509.181558,49.18176,495.0,16.668079,92.5,716.320392,38.6,48.433825,5.545676,57.107643,21.805,2002-01-08


In [10]:
#getting the info on the dataframe again
df_price_close_corr.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4907 entries, 2002-01-02 to 2021-12-30
Data columns (total 13 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   BAC.N    4907 non-null   Float64
 1   BARC.L   4907 non-null   Float64
 2   BNPP.PA  4907 non-null   Float64
 3   C.N      4907 non-null   Float64
 4   CAGR.PA  4907 non-null   Float64
 5   GS       4907 non-null   Float64
 6   HSBA.L   4907 non-null   Float64
 7   JPM.N    4907 non-null   Float64
 8   MS.N     4907 non-null   Float64
 9   SAN.MC   4907 non-null   Float64
 10  SOGN.PA  4907 non-null   Float64
 11  WFC.N    4907 non-null   Float64
 12  Date1    4907 non-null   object 
dtypes: Float64(12), object(1)
memory usage: 594.2+ KB


In [11]:
# convert data from dataframe into a vector of lists
records_to_insert = df_price_close_corr.values.tolist()
records_to_insert

[[31.48,
  509.406461,
  48.362064,
  510.1,
  15.947297,
  91.07,
  706.734596,
  36.35,
  46.168062,
  5.593177,
  55.256933,
  21.72,
  '2002-01-02'],
 [31.445,
  514.579242,
  48.844238,
  510.5,
  15.974326,
  92.45,
  724.163316,
  37.32,
  47.505782,
  5.676303,
  56.402611,
  21.8,
  '2002-01-03'],
 [31.935,
  517.727891,
  49.229977,
  520.0,
  15.938287,
  96.75,
  731.134804,
  39.0,
  49.863513,
  5.711928,
  58.209257,
  21.855,
  '2002-01-04'],
 [31.555,
  511.430593,
  49.085325,
  514.3,
  16.379766,
  95.75,
  724.163316,
  38.9,
  49.487279,
  5.462551,
  56.622934,
  21.89,
  '2002-01-07'],
 [31.035,
  509.181558,
  49.18176,
  495.0,
  16.668079,
  92.5,
  716.320392,
  38.6,
  48.433825,
  5.545676,
  57.107643,
  21.805,
  '2002-01-08'],
 [30.975,
  515.029049,
  50.242543,
  497.9,
  16.668079,
  93.75,
  700.198826,
  38.71,
  48.818419,
  5.492238,
  57.416095,
  21.695,
  '2002-01-09'],
 [31.0,
  517.278084,
  50.001456,
  498.5,
  16.397785,
  93.0,
  691.048

In [13]:
#Compute log returns
log_returns = np.log(df_price_close/df_price_close.shift(1)).dropna()
log_returns

CLOSE,BAC.N,BARC.L,BNPP.PA,C.N,CAGR.PA,GS,HSBA.L,JPM.N,MS.N,SAN.MC,SOGN.PA,WFC.N
Date,,,,,,,,,,,,
2002-01-03,-0.001112,0.010103,0.009921,0.000784,0.001693,0.01504,0.024362,0.026335,0.028563,0.014753,0.020522,0.003676
2002-01-04,0.015463,0.0061,0.007866,0.018438,-0.002259,0.045462,0.009581,0.044032,0.048438,0.006256,0.031529,0.00252
2002-01-07,-0.011971,-0.012238,-0.002943,-0.011022,0.027323,-0.01039,-0.009581,-0.002567,-0.007574,-0.044641,-0.02763,0.0016
2002-01-08,-0.016616,-0.004407,0.001963,-0.038249,0.017449,-0.034532,-0.010889,-0.007742,-0.021517,0.015103,0.008524,-0.003891
2002-01-09,-0.001935,0.011419,0.021339,0.005841,0.0,0.013423,-0.022763,0.002846,0.007909,-0.009683,0.005387,-0.005057
...,...,...,...,...,...,...,...,...,...,...,...,...
2021-12-20,-0.016544,-0.012464,0.004441,-0.021389,-0.009204,-0.027052,-0.008228,-0.018153,-0.018286,-0.007415,-0.010695,-0.023208
2021-12-21,0.023584,0.031199,0.042338,0.018698,0.0231,0.023168,0.007666,0.01387,0.024854,0.020304,0.026269,0.020284
2021-12-22,0.002035,0.008538,0.00829,0.003698,0.006387,0.005271,-0.001349,0.0039,0.005406,0.0016,0.010929,0.004591


In [20]:
#Calculate daily standard deviation of returns
daily_std = log_returns.std()
daily_std

CLOSE
BAC.N      0.028832
BARC.L     0.028833
BNPP.PA    0.024258
C.N        0.031192
CAGR.PA    0.025409
GS         0.021984
HSBA.L     0.016351
JPM.N      0.023878
MS.N       0.029609
SAN.MC     0.022433
SOGN.PA    0.027299
WFC.N      0.024397
dtype: float64

In [21]:
annualized_vol = daily_std * np.sqrt(252)
annualized_vol*100

CLOSE
BAC.N      45.769622
BARC.L     45.770786
BNPP.PA    38.507602
C.N        49.516410
CAGR.PA    40.335250
GS         34.898407
HSBA.L     25.955645
JPM.N      37.905630
MS.N       47.003118
SAN.MC     35.610560
SOGN.PA    43.335663
WFC.N      38.729692
dtype: float64

In [50]:
fig = make_subplots(rows=6, cols=2)

trace0 = go.Histogram(x=log_returns['BAC.N'], name='Bank of America')
trace1 = go.Histogram(x=log_returns['BARC.L'], name='Barclays')
trace2 = go.Histogram(x=log_returns['BNPP.PA'], name='BNP')
trace3 = go.Histogram(x=log_returns['C.N'], name='City Group')
trace4 = go.Histogram(x=log_returns['CAGR.PA'], name = 'Credit Agricole')
trace5 = go.Histogram(x=log_returns['GS'], name = 'Goldman Sachs')
trace6 = go.Histogram(x=log_returns['HSBA.L'], name = 'HSBC')
trace7 = go.Histogram(x=log_returns['JPM.N'], name = 'JP Morgan')
trace8 = go.Histogram(x=log_returns['MS.N'], name = 'Morgan Stanley')
trace9 = go.Histogram(x=log_returns['SAN.MC'], name = 'Banco Santander')
trace10 = go.Histogram(x=log_returns['SOGN.PA'], name = 'Societe Generale')
trace11 = go.Histogram(x=log_returns['WFC.N'], name = 'Wells Fargo')

fig.append_trace(trace0, 1, 1)
fig.append_trace(trace1, 1, 2)
fig.append_trace(trace2, 2, 1)
fig.append_trace(trace3, 2, 2)
fig.append_trace(trace4, 3, 1)
fig.append_trace(trace5, 3, 2)
fig.append_trace(trace6, 4, 1)
fig.append_trace(trace7, 4, 2)
fig.append_trace(trace8, 5, 1)
fig.append_trace(trace9, 5, 2)
fig.append_trace(trace10, 6, 1)
fig.append_trace(trace11, 6, 2)

fig.update_layout(autosize=False, width=600, height=600, title="Frequency of log returns",
                 xaxis=dict(title='BoA Annualized Vol: '+str(np.round(annualized_vol['BAC.N']*100,1))))

fig.show()

In [ ]:
#Trailing volatility over time
Trading_days = 60
volatility = log_returns.rolling(window=Trading_Days)

In [ ]:
#Calculate average cumulative returns

#: Dictionaries that I'm going to be storing calculated data in 
all_returns = {}
all_std_devs = {}
total_sample_size = {} 

#: Create our range of day_numbers that will be used to calculate returns
starting_point = 30
#: Looking from -starting_point till +starting_point which creates our timeframe band
day_numbers = [i for i in range(-starting_point, starting_point)]

for day_num in day_numbers:
    
    #: Reset our returns and sample size each iteration
    returns = []
    sample_size = 0
    
    #: Get the return compared to t=0
    for date, row in ev_data.iterrows():
        sid = row.symbol
        date = row.event_date
        